## Data Challenge - Weather in Australia

In this section, we want you to use the skills from previous labs to extract interesting insights from the "Rain in Australia" dataset. The dataset can be found in [here](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) in Kaggle. You will need to sign up on Kaggle, if you don't have an account yet. If you are interested about data, you must have a Kaggle acount. So if you don't have one yet, consider this a great tip!

To help you get started, this notebook provides a few basic code. Here, I downloaded the data and uploaded it in a folder called weather_aus in my databricks account.

In [2]:
# Similar to SparkContext, for SparkSQL you need a SparkSession
from pyspark.sql import SparkSession
# Also all the functions (select, where, groupby) needs to be imported
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
# instantiate spark session
spark = SparkSession.builder.getOrCreate()

In [4]:
# read weather data
weather_df = spark.read.csv("/FileStore/tables/weather_aus/weatherAUS.csv", header=True)

In [5]:
# take a sneak peak into the data 
display(weather_df.take(10))

Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,Month-Year
2008-12-01T00:00:00.000+0000,Albury,13.4,22.9,0.6,NA,NA,W,44,W,WNW,20,24,71,22,1007.7,1007.1,8,NA,16.9,21.8,No,0,No,2008-12
2008-12-02T00:00:00.000+0000,Albury,7.4,25.1,0,NA,NA,WNW,44,NNW,WSW,4,22,44,25,1010.6,1007.8,NA,NA,17.2,24.3,No,0,No,2008-12
2008-12-03T00:00:00.000+0000,Albury,12.9,25.7,0,NA,NA,WSW,46,W,WSW,19,26,38,30,1007.6,1008.7,NA,2,21,23.2,No,0,No,2008-12
2008-12-04T00:00:00.000+0000,Albury,9.2,28,0,NA,NA,NE,24,SE,E,11,9,45,16,1017.6,1012.8,NA,NA,18.1,26.5,No,1,No,2008-12
2008-12-05T00:00:00.000+0000,Albury,17.5,32.3,1,NA,NA,W,41,ENE,NW,7,20,82,33,1010.8,1006,7,8,17.8,29.7,No,0.2,No,2008-12
2008-12-06T00:00:00.000+0000,Albury,14.6,29.7,0.2,NA,NA,WNW,56,W,W,19,24,55,23,1009.2,1005.4,NA,NA,20.6,28.9,No,0,No,2008-12
2008-12-07T00:00:00.000+0000,Albury,14.3,25,0,NA,NA,W,50,SW,W,20,24,49,19,1009.6,1008.2,1,NA,18.1,24.6,No,0,No,2008-12
2008-12-08T00:00:00.000+0000,Albury,7.7,26.7,0,NA,NA,W,35,SSE,W,6,17,48,19,1013.4,1010.1,NA,NA,16.3,25.5,No,0,No,2008-12
2008-12-09T00:00:00.000+0000,Albury,9.7,31.9,0,NA,NA,NNW,80,SE,NW,7,28,42,9,1008.9,1003.6,NA,NA,18.3,30.2,No,1.4,Yes,2008-12
2008-12-10T00:00:00.000+0000,Albury,13.1,30.1,1.4,NA,NA,W,28,S,SSE,15,11,58,27,1007,1005.7,NA,NA,20.1,28.2,Yes,0,No,2008-12


In [6]:
# convert the date column to datetime instead of string
weather_df = weather_df.withColumn('Date', unix_timestamp(weather_df.Date, 'yyyy-MM-dd').cast(TimestampType()))

In [7]:
TimestampType

Out[6]: pyspark.sql.types.TimestampType

In [8]:
# create a column with only Month and Year using the concat spark sql function
weather_df = weather_df.withColumn('Month-Year',concat(year(weather_df.Date),lit('-'),month(weather_df.Date)))

In [9]:
# take a look at our new column
display(weather_df.take(10))

Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,Month-Year
2008-12-01T00:00:00.000+0000,Albury,13.4,22.9,0.6,NA,NA,W,44,W,WNW,20,24,71,22,1007.7,1007.1,8,NA,16.9,21.8,No,0,No,2008-12
2008-12-02T00:00:00.000+0000,Albury,7.4,25.1,0,NA,NA,WNW,44,NNW,WSW,4,22,44,25,1010.6,1007.8,NA,NA,17.2,24.3,No,0,No,2008-12
2008-12-03T00:00:00.000+0000,Albury,12.9,25.7,0,NA,NA,WSW,46,W,WSW,19,26,38,30,1007.6,1008.7,NA,2,21,23.2,No,0,No,2008-12
2008-12-04T00:00:00.000+0000,Albury,9.2,28,0,NA,NA,NE,24,SE,E,11,9,45,16,1017.6,1012.8,NA,NA,18.1,26.5,No,1,No,2008-12
2008-12-05T00:00:00.000+0000,Albury,17.5,32.3,1,NA,NA,W,41,ENE,NW,7,20,82,33,1010.8,1006,7,8,17.8,29.7,No,0.2,No,2008-12
2008-12-06T00:00:00.000+0000,Albury,14.6,29.7,0.2,NA,NA,WNW,56,W,W,19,24,55,23,1009.2,1005.4,NA,NA,20.6,28.9,No,0,No,2008-12
2008-12-07T00:00:00.000+0000,Albury,14.3,25,0,NA,NA,W,50,SW,W,20,24,49,19,1009.6,1008.2,1,NA,18.1,24.6,No,0,No,2008-12
2008-12-08T00:00:00.000+0000,Albury,7.7,26.7,0,NA,NA,W,35,SSE,W,6,17,48,19,1013.4,1010.1,NA,NA,16.3,25.5,No,0,No,2008-12
2008-12-09T00:00:00.000+0000,Albury,9.7,31.9,0,NA,NA,NNW,80,SE,NW,7,28,42,9,1008.9,1003.6,NA,NA,18.3,30.2,No,1.4,Yes,2008-12
2008-12-10T00:00:00.000+0000,Albury,13.1,30.1,1.4,NA,NA,W,28,S,SSE,15,11,58,27,1007,1005.7,NA,NA,20.1,28.2,Yes,0,No,2008-12


In [10]:
# count number of missing MaxTemp
weather_df.filter(weather_df.MaxTemp == 'NA').count()

Out[9]: 322

In [11]:
# count number of missing MinTemp
weather_df.filter(weather_df.MinTemp == 'NA').count()

Out[10]: 637

In [12]:
# remove NA and then group by month and year and create a dataframe with different aggregation of temperatures
# you will notice I am taking the minimum of the Date, which should be the first of the month. I am keeping this to make plotting easy, as most plotting library accepts datetime.
temp_agg_df = weather_df.filter(weather_df.MinTemp != 'NA').filter(weather_df.MaxTemp != 'NA').groupBy('Month-Year').agg(min(weather_df.Date).alias('firstDate'), min(weather_df.MinTemp).alias('minTemp'), max(weather_df.MaxTemp).alias('maxTemp'))

## Visualization
You can use plotly, matplotlib, seaborn, and other libraries in python to visualize the results in the temp_agg_df. Since we are using databricks, we will simply use the display function to plot the line chart min and max temperature per month and year.

In [14]:
display(temp_agg_df)

Month-Year,firstDate,minTemp,maxTemp
2016-7,2016-07-01T00:00:00.000+0000,-0.1,9.9
2014-1,2014-01-01T00:00:00.000+0000,10,47.3
2013-4,2013-04-01T00:00:00.000+0000,-0.3,9.6
2016-9,2016-09-01T00:00:00.000+0000,-0.3,9.9
2013-12,2013-12-01T00:00:00.000+0000,-0.4,45.1
2010-11,2010-11-01T00:00:00.000+0000,-0.1,9.4
2010-5,2010-05-01T00:00:00.000+0000,-0.3,9.7
2012-1,2012-01-01T00:00:00.000+0000,-1.3,43
2010-1,2010-01-01T00:00:00.000+0000,-0.6,7.6
2011-9,2011-09-01T00:00:00.000+0000,-0.1,9.9


Do you see any interesting observations in the chart above? 

Now it is your turn to explore this dataset and find more interesting insights from it. Please use this notebook as a starter and keep digging forward.